In [13]:
from selenium.webdriver import Chrome
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
import joblib
import pandas as pd
import os

# 1. Скачиваем с официального сайта Selenium веб-драйвер удобного вам браузера и прописываем к нему путь. 

In [27]:
driver = Chrome(executable_path='/Users/nikolaevaanna/WebDriver/bin/chromedriver')

# 2. Функция ниже вытаскивает нужный текст из tei-разметки реплики одного персонажа (пока исключаем комментарии к сценам, которые находятся внутри тега "tei-stage")

In [3]:
def get_speaker_and_text(speaker):
    
    # так получаем имя персонажа, если есть:
    name = speaker.find("tei-speaker").text.strip(".") if speaker.find("tei-speaker") else "" 
                
    # затем удаляем это имя, а также комментарии к сцене (когда кто-то заходит, выходит, чихает, улыбается и т.п.)
    _ = [name.decompose() for name in speaker("tei-speaker")]
    _ = [stage_note.decompose() for stage_note in speaker("tei-stage")]
        
    # потом вытаскиваем текст из всевозможных тегов (пока я выявила такие, но, может, еще есть)
    text = ""
    if speaker.find("tei-p", {"data-origname": "p"}):
        text += " ".join([i.text.strip() for i in speaker("tei-p", {"data-origname": "p"})])
    if speaker.find("tei-lg"):
        text += " ".join([i.text.strip() for i in speaker("tei-lg")])
    if speaker.find("tei-l"):
        text += " ".join([i.text.strip() for i in speaker("tei-l")])
        
    return (name, text) # возвращаем имя персонажа и реплику

# 3. Основная функция проходится по пьесе и собирает цепочки из реплик, которые выглядят примерно так: 

```[('Ефимовна', 'Потолкай-ка, мать, старца! Словно, никак, богу душу отдает.'),
 ('Назаровна', 'Божий человек, а божий человек! Жив ты, аль уж помер?'),
 ('Савва',
  'Зачем помер? Жив, матушка.  Укрой-ка мне,\n            убогонькая, ноги! Вот так. Правую больше. Вот так, матушка. Дай бог здоровья.'),
 ('Назаровна', 'Спи, батюшка.'),
 ('Савва',
  'Какой уж тут сон? Было б терпенье муку эту перенесть, а спанья, матушка, хоть и не\n            надо. Не достоин грешник покой иметь. Это что шумит, богомолочка?'),
  ```

В пьесах могут быть акты, действия, сцены, явления, например:
    - пьеса в четырех действиях
    - пьеса в трех актах (30 сцен)
    - просто 1 сцена
    и т.п. 
Вроде бы парсер разметки, который ниже, может распознавать все возможные паттерны и собирать цепочки сцен. Возможно, парсер работает с ошибками, которые я не отловила, но вроде бы основная структура пьес такая.

In [24]:
def parse_tei(link, driver):
    
    """
    Returns:
    * a list of drama contents (drama_chain) in the following format:

    drama_chain = [scenes]
    scenes = [[(speaker, text), ... ,(speaker, text)], 
              [(speaker, text), ... ,(speaker, text)], 
              ...]
    
    Arguments:
    
    * link - link to the drama page
    * driver - Selenium WebDriver    
    
    """
    
    tei_link = link + '#tei'
    
    driver.get(tei_link)
    WebDriverWait(driver, timeout=20).until(lambda d: d.find_element_by_tag_name("tei-sp"))
    
    soup = BeautifulSoup(driver.page_source, "lxml")
        
    drama_chain = []
    
    acts = soup.findAll("tei-div", {"type":"act"})
    
    if len(acts) != 0: 
        # если есть акты
        for act in acts:
            act_chain = []
            scenes = act.findAll("tei-div", {"type":"scene"})
            if len(scenes) != 0:
                # если есть сцены
                for scene in scenes:
                    speakers = scene("tei-sp")
                    scene_chain = []
                    for speaker in speakers:    
                        scene_chain.append(get_speaker_and_text(speaker))
                    drama_chain.append(scene_chain)

            else: 
                # если нет сцен
                speakers = act("tei-sp")
                for speaker in speakers:
                    act_chain.append(get_speaker_and_text(speaker))
                drama_chain.append(act_chain)   
                
    else: 
        # если нет актов
        scenes = soup.findAll("tei-div", {"type":"scene"})
        
        if len(scenes) != 0: 
            # если нет актов, но есть сцены
            for scene in scenes:
                speakers = scene.findAll("tei-sp")
            
                scene_chain = []
            
                for speaker in speakers:
                    scene_chain.append(get_speaker_and_text(speaker))
                
                drama_chain.append(scene_chain)
        else: 
            # если нет ни актов, ни сцен
            speakers = soup.findAll("tei-sp")
            chain = []
            for speaker in speakers:
                chain.append(get_speaker_and_text(speaker))
            drama_chain.append(chain)
    
    title = link.split("/")[-1]
    # сохраним каждую пьесу в pickle
    joblib.dump(drama_chain, "drama" + os.sep + title + ".pkl") 
    
    print(title + " successfully parsed!")
    
    # return drama_chain
    return drama_chain

In [26]:
driver.quit()

# Попробуем распарсить все ссылки

In [9]:
df = pd.read_csv("dracor-rus-list.csv")
df.head()

,authors,title,link,written,premiered,printed,source,id
0,"Андреев, Леонид Николаевич",К звёздам,https://dracor.org/rus/andreyev-k-zvezdam,1905.0,1906.0,NaN,Библиотека Максима Мошкова (lib.ru),rus000194
1,"Андреев, Леонид Николаевич",Мысль,https://dracor.org/rus/andreyev-mysl,1913.0,1914.0,1914.0,Библиотека Максима Мошкова (lib.ru),rus000137
2,"Андреев, Леонид Николаевич",Не убий,https://dracor.org/rus/andreyev-ne-ubiy,1913.0,1913.0,1913.0,Библиотека Максима Мошкова (lib.ru),rus000138
3,"Афиногенов, Александр Николаевич",Машенька,https://dracor.org/rus/afinogenov-mashenka,1940.0,1941.0,1941.0,Wikisource,rus000167
4,"Бабель, Исаак Эммануилович",Мария,https://dracor.org/rus/babel-marija,1934.0,1964.0,1935.0,Wikilivres/Bibliowiki,rus000119


In [10]:
links = list(df.link)
len(links)

210

In [14]:
os.mkdir("drama")

In [19]:
for link in links:
    parse_tei(link=link,
              driver=driver)

andreyev-k-zvezdam successfully parsed!
andreyev-mysl successfully parsed!
andreyev-ne-ubiy successfully parsed!
afinogenov-mashenka successfully parsed!
babel-marija successfully parsed!
babel-zakat successfully parsed!
belsky-skazanie-o-nevidimom-grade-kitezhe successfully parsed!
blok-balaganchik successfully parsed!
blok-korol-na-ploschadi successfully parsed!
blok-neznakomka successfully parsed!
bulgakov-adam-i-eva successfully parsed!
bulgakov-batum successfully parsed!
bulgakov-beg successfully parsed!
bulgakov-dni-turbinyh successfully parsed!
bulgakov-ivan-vasilevich successfully parsed!
bulgakov-kabala-svjatosh successfully parsed!
bulgakov-poloumnyj-zhurden successfully parsed!
bulgakov-poslednie-dni successfully parsed!
bulgakov-vojna-i-mir successfully parsed!
bulgakov-zojkina-kvartira successfully parsed!
glinka-velzen successfully parsed!
gogol-teatralnyi-razezd successfully parsed!
gogol-tjazhba successfully parsed!
gogol-utro-delovogo-cheloveka successfully parsed!
gog

TimeoutException: Message: 


In [28]:
for link in links[73:]:
    parse_tei(link=link,
              driver=driver)

mayakovsky-banja successfully parsed!
mayakovsky-klop successfully parsed!
mayakovsky-misteriya-buff successfully parsed!
merezhkovsky-pavel-pervyj successfully parsed!
naydyonov-deti-vanjushina successfully parsed!
nekrasov-akter successfully parsed!
nekrasov-junost-lomonosova successfully parsed!
nekrasov-shila-v-meshke-ne-utaish successfully parsed!
nekrasov-zabrakovannye successfully parsed!
nikolev-samolyubivyj-stihotvorec successfully parsed!
ozerov-dmitrij-donskoj successfully parsed!
ostrovsky-neozhidannyj-sluchaj successfully parsed!
ostrovsky-dmitrij-samozvanets-i-vasilij-shujskij successfully parsed!
ostrovsky-nevolnicy successfully parsed!
ostrovsky-dohodnoe-mesto successfully parsed!
ostrovsky-poslednyaya-zhertva successfully parsed!
ostrovsky-goryachee-serdce successfully parsed!
ostrovsky-svoi-sobaki-gryzutsya-chuzhaya-ne-pristavaj successfully parsed!
ostrovsky-pozdnyaya-lyubov successfully parsed!
ostrovsky-trudovoj-hleb successfully parsed!
ostrovsky-tyazhelye-dni suc

TimeoutException: Message: 


In [33]:
for link in links[159:]:
    parse_tei(link=link,
              driver=driver)

sukhovo-kobylin-delo successfully parsed!
sukhovo-kobylin-smert-tarelkina successfully parsed!
sukhovo-kobylin-svadba-krechinskogo successfully parsed!
tolstoy-smert-ioanna-groznogo successfully parsed!
tolstoy-tsar-boris successfully parsed!
tolstoy-tsar-fedor-ioannovich successfully parsed!
tolstoy-vlast-tmy successfully parsed!
tolstoy-zhivoy-trup successfully parsed!
tretyakov-protivogazy successfully parsed!
turgenev-gde-tonko-tam-i-rvetsja successfully parsed!
turgenev-holostjak successfully parsed!
turgenev-mesjats-v-derevne successfully parsed!
turgenev-nahlebnik successfully parsed!
turgenev-neostorozhnost successfully parsed!
turgenev-provintsialka successfully parsed!
turgenev-razgovor-na-bolshoj-doroge successfully parsed!
turgenev-vecher-v-sorrente successfully parsed!
turgenev-zavtrak-u-predvoditelja successfully parsed!
fonvizin-brigadir successfully parsed!
fonvizin-korion successfully parsed!
fonvizin-nedorosl successfully parsed!
fonvizin-vybor-guvernera successfully 

In [36]:
files = ["drama" + os.sep + i for i in os.listdir("drama")]

In [49]:
for i in files:
    drama = joblib.load(i)
    for j in drama:
        for k in j:
            if k[1] == "" or None:
                print(i)
#    print(len(drama))

drama/gogol-revizor.pkl
drama/gogol-revizor.pkl
drama/bulgakov-poslednie-dni.pkl
drama/bulgakov-poslednie-dni.pkl
drama/bulgakov-poslednie-dni.pkl
drama/bulgakov-poslednie-dni.pkl
drama/lermontov-strannyj-chelovek.pkl
drama/kokoshkin-vospitanie.pkl
drama/tolstoy-zhivoy-trup.pkl
drama/lermontov-menschen-und-leidenschaften.pkl
drama/lermontov-menschen-und-leidenschaften.pkl
drama/lermontov-menschen-und-leidenschaften.pkl
drama/lermontov-menschen-und-leidenschaften.pkl
drama/lermontov-menschen-und-leidenschaften.pkl
drama/ostrovsky-groza.pkl
drama/ostrovsky-groza.pkl
drama/tretyakov-protivogazy.pkl
drama/afinogenov-mashenka.pkl
drama/knyazhnin-chudaki.pkl


In [101]:
# drama = joblib.load("drama/bulgakov-poslednie-dni.pkl") - тут ошибки в самой разметке - в цепочках 1 и 2. Вместо tei-p tei-stage
# drama = joblib.load("drama/kokoshkin-vospitanie.pkl") - тоже ошибка в разметке, в 18-й цепочке
# drama = joblib.load("drama/tolstoy-zhivoy-trup.pkl") - кривовато парсится пара первых строчек в 48 цепочке
# drama = joblib.load("drama/ostrovsky-groza.pkl") - c 25-й цепочки поехала разметка - неясно, кто говорит и т.п. Надо фиксить 
# drama = joblib.load("drama/afinogenov-mashenka.pkl") - сомнительная разметка в одном месте в цепочке 4
drama = joblib.load("drama/knyazhnin-chudaki.pkl") # в одном месте ошибка в парсинге - там не захватился текст tei-l 
len(drama)

44

In [102]:
for i,j in enumerate(drama):
    for k in j:
        if k[1] == "" or None:
            print(i)

32


In [103]:
drama[32]

[('Высонос',
  'Я так сердит, что свет мне кажется, как ночь. Надеяся на то, что глупый этот струсит, Намерен я его на поединок звать... Ну, ежели его черт храбрости укусит И если вместо он того, чтобы дрожать, Приободрившися, не вздумает бежать?.. Пропал я!.. Но тому быть, кажется, не можно. Слыхал я: филозо́ф не любит умирать. Притом же поступил я очень осторожно: Бумаги десть на грудь взложил я вместо лат.'),
 ('Пролаз', 'Зачем ты здесь один, любезный друг и брат?'),
 ('Высонос',
  'Один? любезный друг и брат! — о! он робеет! Коль чести правила бездельник разумеет...'),
 ('Пролаз', 'Ну что ж?'),
 ('Высонос',
  'Ну, только... Черт его пускай возьмет! Он смотрит, будто бы меня совсем сожрет.'),
 ('Пролаз',
  'Его мне бледный вид добра не предвещает: Он, кажется, меня, как репку, искусает.'),
 ('Высонос',
  'Ну, милый Высонос! приободрись, дружок! Он храбрым кажется, быть может, только с рожи. Ты знаешь ли, на что твои дела похожи?'),
 ('Пролаз', 'На что?'),
 ('Высонос', 'На то, за что